## 加载工具包

In [3]:
import time
import re
import math
import numpy as np
from six.moves import urllib
import tensorflow as tf

## 定义网络结构

参看VGG网络结构，适当减少了参数尺寸。

In [35]:
#Below code was referred to http://danijar.com/structuring-your-tensorflow-models/
import functools
TOWER_NAME = 'tower'
def define_scope(function):
    attribute = '_cache_' + function.__name__

    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
           # with tf.variable_scope(function.__name__):
                setattr(self, attribute, function(self))
        return getattr(self, attribute)

    return decorator

In [58]:
#Define dog cat classification model
class imageClassification:
    '''Define a basic model for image classification, the model
    Provides graph structure of tensorflow'''
    
    def __init__(self, image_holder, label_holder, keep_prob, batch_size=32, num_class=1):
        self.image_holder = image_holder
        self.label_holder = label_holder
        self.num_class = num_class
        self.weights = []
        self.batch_size = batch_size
        self.keep_prob = keep_prob
        self.prediction
        self.optimizer
        self.accuracy
        self.correct_num
        print('Initializing Image Classification Model!') 
    
    @define_scope
    def sigmoid(self):
        '''Calculate final softmax logits of 
        convolutional neural network'''
        logits = self.inference
        return tf.nn.sigmoid(logits)
        
    @define_scope
    def prediction(self):
        '''Calculate final softmax logits of 
        convolutional neural network'''
        sigmoid = self.sigmoid
        predictions = sigmoid > 0.5
        return tf.cast(predictions, tf.int32)
    
    @define_scope
    def cost(self):
        #labels = tf.one_hot(self.label_holder, self.num_class, 1, 0)
        labels = tf.cast(self.label_holder, tf.float32)
        #Cross entropy
        #cross_entropy = -tf.reduce_mean(labels*
                                       #tf.log(self.prediction), name='cross_entropy')
        cross_entropy = -tf.reduce_mean(labels*tf.log(self.sigmoid) + (1-labels)*tf.log(1-self.sigmoid))
        
        #Regularization
        l2_loss = cross_entropy
        for i in range(len(self.weights)):
                l2_loss += tf.nn.l2_loss(self.weights[i])
        
        return l2_loss
    
    @define_scope
    def optimizer(self):
        '''Define cross entropy loss function and regularization'''      
        l2_loss = self.cost
        #Decaying Learning Rate
        cur_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
        starter_learning_rate = 0.008
        learning_rate = tf.train.exponential_decay(starter_learning_rate, cur_step, 1000, 0.96, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        #optimizer = tf.train.AdamOptimizer(1e-4)
        return optimizer.minimize(l2_loss, global_step=cur_step)
    
    @define_scope
    def accuracy(self):
        '''Calculate accuracy for each epoch of training'''
        #labels = tf.one_hot(self.label_holder, self.num_class, 1, 0)
        correct_prediction = tf.equal(self.label_holder, 
                                      self.prediction)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        return accuracy
    
    @define_scope
    def correct_num(self):
        '''Count correct predictions for testing part'''
        #labels = tf.one_hot(self.label_holder, self.num_class, 1, 0)
        correct_prediction = tf.equal(self.label_holder, 
                                      self.prediction)
        correct_ones = tf.reduce_sum(tf.cast(correct_prediction, tf.float32))
        return correct_ones
    
    @define_scope
    def inference(self):
        """Build the clasification model.
        Args:
        images: Images returned from distorted_inputs() or inputs().
        Returns:
        Logits.
        """
        # We instantiate all variables using tf.get_variable() instead of
        # tf.Variable() in order to share variables across multiple GPU training runs.
        # If we only ran this model on a single GPU, we could simplify this function
        # by replacing all instances of tf.get_variable() with tf.Variable().
        #
        # conv1
        with tf.variable_scope('conv1') as scope:
            kernel = self._variable_with_weight_decay('weights',
                                                 shape=[3, 3, 3, 64],
                                                 wd=0.0)
            conv = tf.nn.conv2d(self.image_holder, kernel, [1, 1, 1, 1], padding='SAME')
            biases = self._variable_on_cpu('biases', [64], tf.constant_initializer(0.0))
            pre_activation = tf.nn.bias_add(conv, biases)
            conv1 = tf.nn.relu(pre_activation, name=scope.name)
            self._activation_summary(conv1)
        # pool1
        pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                               padding='SAME', name='pool1')

        
        # conv2
        with tf.variable_scope('conv2') as scope:
            kernel = self._variable_with_weight_decay('weights',
                                                 shape=[3, 3, 64, 128],                                                 
                                                 wd=0.0)
            conv = tf.nn.conv2d(pool1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = self._variable_on_cpu('biases', [128], tf.constant_initializer(0.1))
            pre_activation = tf.nn.bias_add(conv, biases)
            conv2 = tf.nn.relu(pre_activation, name=scope.name)
            self._activation_summary(conv2)      

        # pool2
        pool2 = tf.nn.max_pool(conv2, ksize=[1, 3, 3, 1],
                               strides=[1, 2, 2, 1], padding='SAME', name='pool2')
        
        # conv3
        with tf.variable_scope('conv3') as scope:
            kernel = self._variable_with_weight_decay('weights',
                                                 shape=[3, 3, 128, 256],                                                 
                                                 wd=0.0)
            conv = tf.nn.conv2d(pool2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = self._variable_on_cpu('biases', [256], tf.constant_initializer(0.1))
            pre_activation = tf.nn.bias_add(conv, biases)
            conv3 = tf.nn.relu(pre_activation, name=scope.name)
            self._activation_summary(conv2)      

        # pool2
        pool3 = tf.nn.max_pool(conv3, ksize=[1, 3, 3, 1],
                               strides=[1, 2, 2, 1], padding='SAME', name='pool2')
        
        pool3_dropout = tf.nn.dropout(pool3, self.keep_prob)
        # local3
        with tf.variable_scope('fully_connected1') as scope:
            # Move everything into depth so we can perform a single matrix multiply.
            reshape_tensor = tf.reshape(pool3_dropout, [self.batch_size, -1], name='reshape_tensor')
            dim = reshape_tensor.get_shape()[1].value
            weights = self._variable_with_weight_decay('weights', shape=[dim, 384],
                                                  wd=0.004)
            biases = self._variable_on_cpu('biases', [384], tf.constant_initializer(0.1))

            local3 = tf.nn.relu(tf.matmul(reshape_tensor, weights) + biases, name=scope.name)
            self._activation_summary(local3)
            
        local3_dropout = tf.nn.dropout(local3, self.keep_prob)
        # local4
        with tf.variable_scope('fully_connected2') as scope:
            weights = self._variable_with_weight_decay('weights', shape=[384, 192],
                                                  wd=0.004)
            biases = self._variable_on_cpu('biases', [192], tf.constant_initializer(0.1))
            local4 = tf.nn.relu(tf.matmul(local3_dropout, weights) + biases, name=scope.name)
            self._activation_summary(local4)
            
        local4_dropout = tf.nn.dropout(local4, self.keep_prob)
        
        # linear layer(WX + b),
        # We don't apply softmax here because
        # tf.nn.sparse_softmax_cross_entropy_with_logits accepts the unscaled logits
        # and performs the softmax internally for efficiency.
        with tf.variable_scope('softmax_linear') as scope:
            weights = self._variable_with_weight_decay('weights', [192, self.num_class],
                                                  wd=0.0)
            biases = self._variable_on_cpu('biases', [self.num_class],
                                      tf.constant_initializer(0.0))
            softmax_linear = tf.add(tf.matmul(local4_dropout, weights), biases, name=scope.name)
            self._activation_summary(softmax_linear)
        return softmax_linear
    
    #def _conv_layer(self, )
    
    def _activation_summary(self, x):
        """Helper to create summaries for activations.
        Creates a summary that provides a histogram of activations.
        Creates a summary that measures the sparsity of activations.
        Args:
        x: Tensor
        Returns:
        nothing
        """
        # Remove 'tower_[0-9]/' from the name in case this is a multi-GPU training
        # session. This helps the clarity of presentation on tensorboard.
        tensor_name = re.sub('%s_[0-9]*/' % TOWER_NAME, '', x.op.name)
        tf.summary.histogram(tensor_name + '/activations', x)
        tf.summary.scalar(tensor_name + '/sparsity',
                          tf.nn.zero_fraction(x))
    
    def _variable_on_cpu(self, name, shape, initializer):
        """Helper to create a Variable stored on CPU memory.
        Args:
        name: name of the variable
        shape: list of ints
        initializer: initializer for Variable
        Returns:
        Variable Tensor
        """
        with tf.device('/cpu:0'):
            with tf.variable_scope('weights'):
                dtype = tf.float32
                var = tf.get_variable(initializer=initializer(shape), dtype=dtype, name=name)
            #var = tf.get_variable(name, shape, initializer=initializer, dtype=dtype)
        return var

    def _variable_with_weight_decay(self, name, shape, wd):
        """Helper to create an initialized Variable with weight decay.
        Note that the Variable is initialized with a truncated normal distribution.
        A weight decay is added only if one is specified.
        Args:
        name: name of the variable
        shape: list of ints
        stddev: standard deviation of a truncated Gaussian
        wd: add L2Loss weight decay multiplied by this float. If None, weight
        decay is not added for this Variable.
        Returns:
        Variable Tensor
        """
        dtype = tf.float32
        var = self._variable_on_cpu(name, shape,
                               tf.contrib.layers.xavier_initializer())
        if wd is not None:
            weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
            #tf.add_to_collection('losses', weight_decay)
            self.weights.append(weight_decay)
        return var

## 定义输入张量

In [92]:
batch_size = 64
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    image_holder = tf.placeholder(tf.float32, [batch_size, 64, 64, 3])
    label_holder = tf.placeholder(tf.int32, [batch_size, 1])
    keep_prob = tf.placeholder(tf.float32)
    model = imageClassification(image_holder, label_holder, keep_prob, batch_size, num_class =1)
    saver = tf.train.Saver()

Initializing Image Classification Model!


## 处理训练数据和测试数据

In [78]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)
path = 'C:/Users/onlooker/Documents/deeplearning_projects/cats_dogs_classification/dataset'
training_set = train_datagen.flow_from_directory(path+'/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = batch_size,
                                                 class_mode = 'binary')


Found 8000 images belonging to 2 classes.


In [103]:
test_set = test_datagen.flow_from_directory(path+'/test_set', shuffle=False,
                                            target_size = (64, 64),
                                            batch_size = batch_size,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## 训练和测试

In [98]:
num_steps = 2001
epochs = 30
with tf.Session(graph=graph) as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    for _ in range(epochs):
        for step in range(num_steps):
            #产生训练用样本集      
            batch_data, batch_labels = training_set.next()
            batch_labels = np.reshape(batch_labels, [-1,1])
            #数据传递给tensorflow
            feed_dict = {image_holder: batch_data, label_holder : batch_labels, keep_prob: 0.5}
            _, loss = sess.run([model.optimizer, model.cost], feed_dict=feed_dict)
            if step%400 == 0:
                #每50次计算准确度
                #result = sess.run(cross_entropy, feed_dict={x : batch_data, y_ : batch_labels, keep_prob:1})
                print('Cost:', loss) 
                #保存模型
        saver.save(sess, "Model/model.ckpt")


Cost: 1.94332
Cost: 1.81627
Cost: 1.79601
Cost: 1.6962
Cost: 1.69042
Cost: 1.64443
Cost: 1.55749
Cost: 1.49317
Cost: 1.43715
Cost: 1.38808
Cost: 1.32744
Cost: 1.27909
Cost: 1.38868
Cost: 1.25888
Cost: 1.25231
Cost: 1.30282
Cost: 1.26781
Cost: 1.19784
Cost: 1.2011
Cost: 1.07712
Cost: 0.98704
Cost: 1.13203
Cost: 1.09172
Cost: 1.04107
Cost: 0.969746
Cost: 1.00542
Cost: 0.923957
Cost: 0.975079
Cost: 0.95894
Cost: 0.913933
Cost: 0.878574
Cost: 0.988095
Cost: 0.924557
Cost: 0.850962
Cost: 0.804787
Cost: 0.918714
Cost: 0.700105
Cost: 0.923358
Cost: 0.852657
Cost: 0.871645
Cost: 0.772391
Cost: 0.721123
Cost: 0.860832
Cost: 0.676857
Cost: 0.763931
Cost: 0.702629
Cost: 0.759242
Cost: 0.725119
Cost: 0.761793
Cost: 0.75006
Cost: 0.82626
Cost: 0.574702
Cost: 0.631242
Cost: 0.660351
Cost: 0.714003
Cost: 0.665653
Cost: 0.722263
Cost: 0.719482
Cost: 0.741925
Cost: 0.723552
Cost: 0.618318
Cost: 0.653924
Cost: 0.724325
Cost: 0.696895
Cost: 0.578147
Cost: 0.501305
Cost: 0.497982
Cost: 0.478435
Cost: 0.69

In [104]:
with tf.Session(graph=graph) as sess:
    saver.restore(sess, "./Model/model.ckpt") # 注意此处路径前添加"./" 
    print('Testing Result.....')  
    count = 0
    for i in range(30):
        batch_data, batch_labels = test_set.next()
        #print(batch_data.shape)
        #print(i)
        batch_labels = np.reshape(batch_labels, [-1,1])
        feed_dict = {image_holder: batch_data, label_holder : batch_labels, keep_prob: 1}
        c = sess.run(model.correct_num, feed_dict=feed_dict)
        count += c
    result = count*1.0/(30*64)
    print('Accuracy:', result)

INFO:tensorflow:Restoring parameters from ./Model/model.ckpt
Testing Result.....
Accuracy: 0.8984375
